# UAV product workflow

In order for UAV data to be published in the DEPWS spatial library UAV data procedures require that the data has been processed using the approved methodology, and that the final product has been composited (where applicable) clipped, renamed with its Meta data updated.

This notebook facilities the processing of UAV images by:
 - Compositing grayscale images form the multispectral camera
 - Clipping images with an extent vector
 - Renaming the output in the required format
 - Updating final product metadata.


## Import modules

In [32]:
import os
import shutil
import gdal
import numpy as np
from osgeo import gdal, gdalconst

In [54]:
site = "gry04"
dir_ = "mosaic_n"
directory = r"Z:\Scratch\uav\2022"

In [55]:
os.chdir(r"{0}\{1}\{2}".format(directory, site, dir_))

In [56]:
ls

 Volume in drive Z is Data_02 (Sat Imagery) (Disk4)
 Volume Serial Number is FABC-DD60

 Directory of Z:\Scratch\uav\2022\gry04\mosaic_n

06/03/2022  03:29 PM    <DIR>          .
06/03/2022  03:29 PM    <DIR>          ..
06/03/2022  12:16 PM               605 p4gh_gry04_n_mosaic_group1.prj
06/03/2022  12:16 PM                76 p4gh_gry04_n_mosaic_group1.tfw
06/03/2022  03:30 PM       215,405,642 p4gh_gry04_n_mosaic_group1.tif
06/03/2022  03:30 PM       103,291,175 p4gh_gry04_n_mosaic_group1.tif.ovr
               4 File(s)    318,697,498 bytes
               2 Dir(s)  1,734,377,357,312 bytes free


## Composite greyscale images

In [36]:
cd Z:\Scratch\Rob\code\draft\uav\uav_composite_image

Z:\Scratch\Rob\code\draft\uav\uav_composite_image


### Composite Multi band UAV images

In [37]:
%run run_composite_multi_bands_image.py \
--directory Z:\Scratch\uav\2022\wmc03\mosaic_n  \
    --outfile Z:\Scratch\uav\2022\wmc03\mosaic_n\p4cj_wmc03_20220402_recnm3.tif

Z:\Scratch\uav\2022\wmc03\mosaic_n\p4cj_wmc03_20220402_recnm3.tif complete


## Clip raster with extent shapefile

In [57]:
os.listdir(r"{0}\{1}\mosaic_n".format(directory, site))

['p4gh_gry04_n_mosaic_group1.prj',
 'p4gh_gry04_n_mosaic_group1.tfw',
 'p4gh_gry04_n_mosaic_group1.tif',
 'p4gh_gry04_n_mosaic_group1.tif.ovr']

In [58]:
if not os.path.isdir(r"{0}\{1}\shp".format(directory, site)):   
    os.mkdir(r"{0}\{1}\shp".format(directory, site))
    
os.listdir(r"{0}\{1}\shp".format(directory, site))

['gry04_extent.CPG',
 'gry04_extent.dbf',
 'gry04_extent.prj',
 'gry04_extent.sbn',
 'gry04_extent.sbx',
 'gry04_extent.shp',
 'gry04_extent.shp.DZ0320.8708.21068.sr.lock',
 'gry04_extent.shp.xml',
 'gry04_extent.shx']

## Multispectral Composite Clip

In [41]:
cd C:\Users\rmcgr\Rangeland_monitoring\image_processing

C:\Users\rmcgr\Rangeland_monitoring\image_processing


In [42]:
%run clip_tool_rasterio.py \
--raster Z:\Scratch\uav\2022\wmc03\mosaic_n\p4cj_wmc03_20220402_recnm3.tif \
    --shpfile Z:\Scratch\uav\2022\wmc03\shp\wmc03_extent.shp \
        --outfile Z:\Scratch\uav\2022\wmc03\p4cj_wmc03_20220402_refnm3.tif

32753 +proj=utm +zone=53 +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs


## RGB Clip

In [59]:
cd C:\Users\rmcgr\Rangeland_monitoring\image_processing

C:\Users\rmcgr\Rangeland_monitoring\image_processing


In [60]:
%run clip_tool_rasterio.py \
--raster Z:\Scratch\uav\2022\gry04\mosaic_n\p4gj_gry04_n_mosaic_group1.tif \
    --shpfile Z:\Scratch\uav\2022\gry04\shp\gry04_extent.shp \
        --outfile Z:\Scratch\uav\2022\gry04\p4gj_gry04_20220525_refnm3.tif

RasterioIOError: Z:\Scratch\uav\2022\gry04\mosaic_n\p4gj_gry04_n_mosaic_group1.tif: No such file or directory

## Metadata update

### All UAV final products

In [45]:
year = '20220402'
pilot = 'Robert McGregor'
processing_date = "20220520"
utm = '53'
image_code = 'p4cj'

In [46]:

image = r"{0}\{1}\{3}_{1}_{2}_refnm3.tif".format(directory, site, year, image_code)
print('Update metadata for: ',  image)
gdal.AllRegister()

inputdataset = gdal.Open(image)

inputdataset.SetMetadataItem("Flight","{0}".format(site))

inputdataset.SetMetadataItem("Flight date","{0}".format(year))
inputdataset.SetMetadataItem("Pilot","{0}".format(pilot))

# platform and sensor
if image[-28:-27] == 'a':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4")
    inputdataset.SetMetadataItem("Sensor","Micasense RedEdge-3")
    bands = "blue, green, red, red edge, nir"
elif image[-28:-27] == 'b':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4")
    inputdataset.SetMetadataItem("Sensor","Micasence RedEdge-M") 
    bands = "blue, green, red, red edge, nir"
elif image[-28:-27] == 'c':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4")
    inputdataset.SetMetadataItem("Sensor","DJI Multi WM331A")
    bands = "blue, green, red, red edge, nir"
elif image[-28:-27] == 'd':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4")
    inputdataset.SetMetadataItem("Sensor","DJI RGB WM331A")  
    bands = "blue, green, red"
elif image[-28:-27] == 'e':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4")
    inputdataset.SetMetadataItem("Sensor","DJI RGB WM331A") 
    bands = "blue, green, red"
elif image[-28:-27] == 'g':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4")
    inputdataset.SetMetadataItem("Sensor","DJI RGB WM331A")
    bands = "blue, green, red"
else:
    pass
    
inputdataset.SetMetadataItem("Processing date","{0}".format(processing_date))
                             
#datum                             
if utm == '52':                     
    inputdataset.SetMetadataItem("Datum","WGS 1984")
    inputdataset.SetMetadataItem("Projection","UTM zone 52S")
    inputdataset.SetMetadataItem("EPSG","32752")
elif utm == '53':
    inputdataset.SetMetadataItem("Datum","WGS 1984")
    inputdataset.SetMetadataItem("Projection","UTM zone 53S")
    inputdataset.SetMetadataItem("EPSG","32753") 
else:
    pass

gcp = image[-7:-6]
if gcp == 'g':
    inputdataset.SetMetadataItem("Geo-rectified","Yes")
elif gcp == 'n':
    inputdataset.SetMetadataItem("Geo-rectified","No")
else:
    pass

inputdataset.SetMetadataItem("Band composition","{0}".format(bands))

inputdataset = None

src = gdal.Open(image, gdalconst.GA_ReadOnly)

print(src.GetMetadata())

Update metadata for:  Z:\Scratch\uav\2022\wmc03\p4cj_wmc03_20220402_refnm3.tif
{'AREA_OR_POINT': 'Area', 'Band composition': 'blue, green, red, red edge, nir', 'Datum': 'WGS 1984', 'EPSG': '32753', 'Flight': 'wmc03', 'Flight date': '20220402', 'Geo-rectified': 'No', 'Pilot': 'Robert McGregor', 'Platform': 'DJI Phantom 4', 'Processing date': '20220520', 'Projection': 'UTM zone 53S', 'Sensor': 'DJI Multi WM331A'}


In [ ]:
shutil.rmtree(r"{0}\{1}\mosaic_n".format(directory, site))

In [ ]:
shutil.rmtree(r"{0}\{1}\mosaic_g".format(directory, site))